---
title: IDs from Wind
order: 12
---

See following notebooks for details:

- [Magnetic field data](./mag.ipynb)
- [State data](./state.ipynb)


References:

- [HPDE.io/Wind](https://hpde.io/NASA/NumericalData/Wind/index.html)
- [Wind Data Sources](https://wind.nasa.gov/data_sources.php)
- [WIND MFI high-resolution data file](https://cdaweb.gsfc.nasa.gov/misc/NotesW.html#WI_H4-RTN_MFI)
- [PySPEDAS](https://pyspedas.readthedocs.io/en/latest/wind.html)

Notes:

- December 2014: Wind Data Notice - Data between September 24 and November 30, 2014 were not included at this definitive data set because of instrumental problem [*](https://omniweb.gsfc.nasa.gov/ftpbrowser/wind_swe_2m.html)

## Setup

Need to run command in shell first as `pipeline` is project-specific command

```{sh}
kedro pipeline create wind
```

In [4]:
# | code-summary: import all the packages needed for the project
# | output: hide
# | export
from ids_finder.pipelines.wind.mag import create_pipeline as create_mag_data_pipeline
from ids_finder.pipelines.wind.state import create_pipeline as create_state_data_pipeline
from ids_finder.pipelines.default.mission import create_combined_data_pipeline

In [ ]:
#| default_exp pipelines/wind/pipeline

In [ ]:
# | export
def create_pipeline(
    sat_id="Wind",
):
    return (
        create_mag_data_pipeline(sat_id)
        + create_state_data_pipeline(sat_id)
        + create_combined_data_pipeline(sat_id)
    )